In [1]:
from ioh_xplainer import explainer
import pandas as pd

from ConfigSpace import ConfigurationSpace
from ConfigSpace.util import generate_grid

cs = ConfigurationSpace({
    "F": (0.05, 2.0),              # Uniform float
    "CR" : (0.05, 2.0),                    # Uniform float
})


de_explainer = explainer(None, 
                 cs , 
                 dims = [5,10,40],#, 10, 20, 40 
                 fids = [1,5,7,13,18,20,23], #,5
                 iids = 10, #20 
                 reps = 5, 
                 sampling_method = "grid",  #or random
                 grid_steps_dict = {},
                 sample_size = None,  #only used with random method
                 budget = 10000, #10000
                 seed = 1,
                 verbose = True)


de_explainer.load_results("df.pkl")
df = de_explainer.performance_stats()
print(df)
#print(de_explainer.stats[5])
#df = de_explainer.stats[5]
print(df.to_latex(index=False,
                  float_format="{:.2f}".format,
                  multicolumn_format = "c"
))  

/home/neocortex/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/neocortex/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/neocortex/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/neocortex/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Using `tqdm.autonotebook.tqdm` in note

AttributeError: 'explainer' object has no attribute 'avg_stats'